In [1]:
import pandas as pd
import geopandas as gp
import fiona
import shapely.geometry as geom

from pathlib import Path

#### Create the ??\_od_distance\_1000\_plus.csv file first using "write_state_csv.ipynb"

In [2]:
ODpath = Path("../data/OD/")
OD_file = ODpath.joinpath("21_od_distance_500.csv.gz")
if OD_file.exists ():
    print ("OD file exist")
else:
    print ("OD file does not exist")
    

OD file exist


In [3]:
df = pd.read_csv(OD_file, dtype={'w_geocode': str,'h_geocode':str, 'w_group_count':str})

In [6]:
len(df)

45065548

#### Identify null records

In [7]:
df_null = df[df.isnull().any(axis=1)]
print ('\nthe number of null records:', "{:,}".format(len(df_null)),'\n\n')
df_null.head()


the number of null records: 45,065,548 




,w_geocode,h_geocode,distance,w_group_count,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,h_lat,h_lon,state
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df[['w_lat', 'w_lon','h_lat', 'h_lon']].head()

,w_lat,w_lon,h_lat,h_lon
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [5]:
df_null = df[df.isnull().any(axis=1)]
print ('\nthe number of null records:', "{:,}".format(len(df_null)),'\n\n')
df_null.head()


the number of null records: 45,065,548 




,w_geocode,h_geocode,distance,w_group_count,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,h_lat,h_lon,state
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_filter = df[(df['distance'] <= 100000)]
len(df_filter)

887367

#### Turn on next block if precision is not set at 3 for lat lng

#### Create a LineString geomotry

In [5]:
df['geometry'] = df.apply(lambda x: geom.LineString([(x['w_lon'], x['w_lat'] ), (x['h_lon'],x['h_lat'])]), axis = 1)

In [6]:
df.drop(['w_lat', 'w_lon', 'h_lat', 'h_lon', 'w_geocode', 'h_geocode'],axis=1, inplace=True)

In [7]:
gdf = gp.GeoDataFrame(df, geometry='geometry')

In [8]:
len(gdf)

3601903

In [9]:
state_id = '06'
outputstate = state_id + '.json'
out_state = ODpath.joinpath(outputstate)

In [10]:
gdf.to_file(out_state, index=None, driver='GeoJSON')

In [11]:
gdf.head()

,distance,w_group_count,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,state,geometry
0,4636,2994,1,0,0,1,0,1,0,0,0,1,6,"LINESTRING (-122.268 37.837, -122.228 37.865)"
1,4214,2994,2,0,1,1,0,0,2,0,0,2,6,"LINESTRING (-122.268 37.837, -122.229 37.86)"
2,4437,2994,1,0,0,1,0,0,1,0,0,1,6,"LINESTRING (-122.268 37.837, -122.224 37.858)"
3,3693,2994,1,0,0,1,0,0,1,0,0,1,6,"LINESTRING (-122.268 37.837, -122.233 37.856)"
4,4116,2994,1,0,1,0,0,0,1,0,0,1,6,"LINESTRING (-122.268 37.837, -122.226 37.855)"


#### Filter data by those earning 1,250 per month or less

In [12]:
gdf_limited_income = gdf[(gdf['SE01'] > 0) & (gdf['distance'] <= 100000)]
len(gdf_limited_income)

KeyError: 'SE01'

In [ ]:
state_id = '21'
outputstate = state_id + '_low_income.json'
out_state = ODpath.joinpath(outputstate)
gdf_limited_income.to_file(out_state, index=None, driver='GeoJSON')

#### Filter data by those earning more than 1,250 and less than 3,333 per month

In [13]:
gdf_limited_income = gdf[(gdf['SE02'] > 0) & (gdf['distance'] <= 100000)]
len(gdf_limited_income)

102278

In [14]:
state_id = '21'
outputstate = state_id + '_medium_income.json'
out_state = ODpath.joinpath(outputstate)
gdf_limited_income.to_file(out_state, index=None, driver='GeoJSON')

#### Filter data by those earning 3,3333 per month or more

In [15]:
gdf_limited_income = gdf[(gdf['SE03'] > 0) & (gdf['distance'] <= 100000)]
len(gdf_limited_income)

138610

In [16]:
state_id = '21'
outputstate = state_id + '_high_income.json'
out_state = ODpath.joinpath(outputstate)
gdf_limited_income.to_file(out_state, index=None, driver='GeoJSON')